# Recurrent neural network

Character-level language model

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg">

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

## Download data

In [2]:
isfile("input.txt") ||
  download("http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt", "input.txt")

true

## Preprocessing

In [3]:
text = collect(readstring("input.txt"))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

68-element Flux.OneHotVector:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [4]:
N = length(alphabet)
seqlen = 50
nbatch = 50

50

In [5]:
Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

1830-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}[Bool[false false … false false; true false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; true false … false false; … ; false false … false false; false false … false false], Bool[false false … false false; false false … false

## Model

In [6]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 128),
    Dense(128, N),
    softmax)

Chain(Recur(LSTMCell(68, 128)), Recur(LSTMCell(128, 128)), Dense(128, 68), NNlib.softmax)

## Loss function

In [7]:
function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

loss (generic function with 1 method)

## Optimizer

In [8]:
opt = ADAM(params(m), 0.01)
evalcb = () -> @show loss(Xs[5], Ys[5])

(::#5) (generic function with 1 method)

## Training

In [9]:
Flux.train!(loss, zip(Xs, Ys), opt, cb=throttle(evalcb, 30))

loss(Xs[5], Ys[5]) = 203.09646060497474 (tracked)
loss(Xs[5], Ys[5]) = 167.96401284221164 (tracked)
loss(Xs[5], Ys[5]) = 167.4380732455794 (tracked)
loss(Xs[5], Ys[5]) = 166.7097275145148 (tracked)
loss(Xs[5], Ys[5]) = 172.0107039964805 (tracked)
loss(Xs[5], Ys[5]) = 159.11203261176658 (tracked)
loss(Xs[5], Ys[5]) = 153.0698478097455 (tracked)
loss(Xs[5], Ys[5]) = 147.56649215170836 (tracked)
loss(Xs[5], Ys[5]) = 142.2601354434028 (tracked)
loss(Xs[5], Ys[5]) = 136.8592694689727 (tracked)
loss(Xs[5], Ys[5]) = 133.02964763432954 (tracked)
loss(Xs[5], Ys[5]) = 130.24398393163756 (tracked)
loss(Xs[5], Ys[5]) = 128.48116252447937 (tracked)
loss(Xs[5], Ys[5]) = 126.07221310477959 (tracked)
loss(Xs[5], Ys[5]) = 125.20465019120205 (tracked)
loss(Xs[5], Ys[5]) = 122.96120997247715 (tracked)
loss(Xs[5], Ys[5]) = 121.63600463262587 (tracked)
loss(Xs[5], Ys[5]) = 120.95785408175782 (tracked)
loss(Xs[5], Ys[5]) = 120.21419281049805 (tracked)
loss(Xs[5], Ys[5]) = 118.57510972650697 (tracked)
loss(X

## Sampling

In [10]:
function sample(m, alphabet, len; temp=1)
    Flux.reset!(m)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, m(onehot(c, alphabet)).data)
    end
    return String(take!(buf))
end

sample (generic function with 1 method)

In [11]:
sample(m, alphabet, 1000) |> println

ee and
My lost slaply vanwillow'd the hard; a fashs as uncleen to exchise,
In rest the cursed for of this rest my which or
will one again? well hespined you, and who morrowite,
Watter, but can meanys to him, on alpeal: and Elyes,
When thou commandend and-younger it lay confesn a cale;
Keep hithing clontion.

First KING HENRY IVELVIIA:
The sther reep, they the rid is prince you mistrand so,
It leave a flends a time, mounded mine Engrialind,
Nothing with an ase tarking! life him,
Their maints: I that tark: for be authing's ridgast:
Brave for a wich indufight indeed soft couse;
I could scorn juttius what before;
Help arm the king bind he lay?

BERTRAM:
What hand of the such to my trullly soft,
As much' honain for our thround meerds of grace him.

PHIMONIUS AFRE:
Then will entrepale, your loved a havasior no treasoathied.

PROSPERO:
What, I came
To ormoral names them we shall best honour, no put wife to her offections comile now,
Let you will, blood,--Her earty.

Second Cable not the lords